# Genetic Algorithms and Finance
Attempted implementation of https://drive.google.com/file/d/0B9-kA56h5JCMMW4tU25HRExWb1U/view?usp=sharing

## Data Gathering

In [45]:
from getKey import *
import quandl
quandl.ApiConfig.api_key = getKey('quandl.key')

#S&P 500 returns
SPY = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH")
SPY["shifted"] = SPY.shift(periods=1)
SPY["Returns"] = (SPY["Value"] - SPY['shifted'])/SPY['shifted']
data = SPY[["Value", "Returns"]]
data = data.dropna()
print(data.head())

            Value   Returns
Date                       
1871-02-01   4.50  0.013514
1871-03-01   4.61  0.024444
1871-04-01   4.74  0.028200
1871-05-01   4.86  0.025316
1871-06-01   4.82 -0.008230


Indicators: SMA, MACD, RSI, Price Oscillator, Larry Williams, Bollinger Bands

In [159]:
#True on the index that a crosses over b
def crossover(a, b):
    a.dropna(inplace = True)
    if not isinstance(b, pd.Series):
        b = pd.Series(b, index = a.index)
    else:
        b.dropna(inplace = True)
    minDate = max(a.index[0], b.index[0])
    maxDate = min(a.index[-1], b.index[-1])
    a = a[minDate:maxDate]
    b = b[minDate:maxDate]
    previousPeriodLessThan = a.shift(1) < b.shift(1) #check if in the previous day, a was less than b
    nextPeriodGreaterThan = a > b #in the day after the previous day ("today") a is greater than b
    out = previousPeriodLessThan & nextPeriodGreaterThan
    return out

#True on the index that a crosses under b
def crossunder(a, b):
    a.dropna(inplace = True)
    if not isinstance(b, pd.Series):
        b = pd.Series(b, index = a.index)
    else:
        b.dropna(inplace = True)
    minDate = max(a.index[0], b.index[0])
    maxDate = min(a.index[-1], b.index[-1])
    a = a[minDate:maxDate]
    b = b[minDate:maxDate]
    #note: names not changed, copied from crossover, but still works
    previousPeriodLessThan = a.shift(1) > b.shift(1) #check if in the previous day, a was greater than b
    nextPeriodGreaterThan = a < b #in the day after the previous day ("today") a is less than b
    out = previousPeriodLessThan & nextPeriodGreaterThan
    return out

In [176]:
import talib
import pandas as pd
close = data.Value
indicators = []

class Indicator:
    def __init__(self, name, buy, sell):
        self.name = name
        self.buy = buy.dropna()
        self.sell = sell.dropna()
        self.minDate = buy.index[0]
        self.maxDate = buy.index[-1]

In [177]:
#SMA
SMA50 = talib.SMA(close, 50)
SMA200 = talib.SMA(close, 200)
buy = crossover(SMA50, SMA200)
sell = crossunder(SMA50, SMA200)
SMA = Indicator("SMA", buy, sell)
indicators.append(SMA)

In [178]:
#MACD
macd, macdsignal, macdhist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
buy = crossover(macd, macdsignal)
sell = crossunder(macd, macdsignal)
MACD = Indicator("MACD", buy, sell)
indicators.append(MACD)

In [179]:
#RSI
rsi = talib.RSI(close, 14)
buy = crossover(rsi, 30)
sell = crossunder(rsi, 70)
RSI = Indicator("RSI", buy, sell)
indicators.append(RSI)

In [ ]:
#Percentage Price Oscillator
ppo = talib.PPO(close, fastperiod=12, slowperiod=26, matype=0)
buy = 

In [182]:
for x in indicators:
    print(x.name)

SMA
MACD
RSI
